# First try on interfacing EnergyPlus and REHO

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from eppy.modeleditor import IDF
import seaborn as sns
import html5lib
from reho.model.reho import *

## Hardcoded "optimization" loop

The goal is to simulate the CELLS building using EnergyPlus. From this simulation, variable of interests will be extracted to run a REHO optimization. From this optimization, installed capacity of technologies will be extracted from the results. Finally, those results will be reused as input into the EnergyPlus simulation. Convergence of the variable of interest will then be studied.

### First `buildings.csv` generation

The minimum variables needed by `REHO` in its buildings.csv files are the following:
- `area_era_m2`
- `id_building` --> set as 1
- `temperature_cooling_supply_C` --> kept as default 12°
- `temperature_cooling_return_C` --> kept as default 17°
- `temperature_heating_supply_C` --> kept as default 65°
- `temperature_heating_return_C` --> kept as default 50°
- `area_facade_m2` 
- `area_roof_solar_m2` --> 100% of roof availability
- `temperature_interior_C` -->  extracted from `.idf` temperature setpoint
- `ratio` --> kept as default 1
- `status` --> kept as default [existing]
- `id_class` --> set from SIA norms individual office:  type III  
- `solar_gain_factor_signature_m2` --> set to default values 0.07
- `thermal_specific_capacity_Wh_m2_K`
- `thermal_transmittance_signature_kW_m2_K` --> extracted from EPlus results

In [51]:
# hardcoded base values for the buildings.csv file
data = {
    'area_era_m2': [53.91],
    'id_building': [1],
    'temperature_cooling_supply_C': [12],
    'temperature_cooling_return_C': [17],
    'temperature_heating_supply_C': [65],
    'temperature_heating_return_C': [50],
    'area_facade_m2': [72.07],
    'area_roof_solar_m2': [53.91],
    'temperature_interior_C': [21],
    'ratio': [1.0],
    'status': ['[existing]'],
    'id_class': ['III'],
    'solar_gain_factor_signature_m2': [0.07],  # not computed yet
    'thermal_specific_capacity_Wh_m2_K': [120],
    'thermal_transmittance_signature_kW_m2_K': [0.169/1000]
}

buildings = pd.DataFrame(data)
display(buildings)

,area_era_m2,id_building,temperature_cooling_supply_C,temperature_cooling_return_C,temperature_heating_supply_C,temperature_heating_return_C,area_facade_m2,area_roof_solar_m2,temperature_interior_C,ratio,status,id_class,solar_gain_factor_signature_m2,thermal_specific_capacity_Wh_m2_K,thermal_transmittance_signature_kW_m2_K
0,53.91,1,12,17,65,50,72.07,53.91,21,1.0,[existing],III,0.07,120,0.000169


In [52]:
## Define the paths to the various inputs
# Specify path to the IDF file
idf_file_path = r".\input_data\CELLS.idf"

# Specify path to the IDD file
idd_file_path = r".\input_data\Energy+.idd" # Energy+ v9.4 IDD

# Specify path to the weather file
weather_file_path = r".\input_data\-_FRIBOURG_MN7.epw"

# Set the idd file
IDF.setiddname(idd_file_path)

# Create an IDF object and load the IDF file
idf = IDF(idf_file_path, weather_file_path)

# Specify path to the EnergyPlus results folder
idf_results_path = r".\output_data"

In [53]:
## Run base EnergyPlus simulation
# readvars = True creates a .cvs of the results
# annual = True forces the simulation to run on an annual basis

idf.run(readvars=True,output_directory=idf_results_path, annual=True, verbose='q', output_prefix='idf1_')

In [54]:
# Steven Firth datetime parser
def parse_energyplus_datetime_string(st, year=2021):
    st=st.strip()
    month=int(st[0:2])
    day=int(st[3:5])
    hour=int(st[7:9])
    minute=int(st[10:12])
    second=(st[13:15])
    if not hour==24:
        dt=pd.Timestamp(year,month,day,hour,minute)
    else:
        hour=0
        dt=pd.Timestamp(year,month,day,hour,minute)
        dt+=pd.Timedelta('1 day')
    return dt

In [55]:
def extract_U_factor(results_path=r'output_data\idf1_tbl.htm'):
    # read EPlus results as html
    df = pd.read_html(results_path, header=0)
    # detect indices where "U-Factor" appears
    indices = []
    numerator = 0
    denominator = 0
    for idx, tables in enumerate(df):
        # Check if "U-factor" appears in the column names
        if any(isinstance(col, str) and 'U-Factor' in col for col in tables.columns):
            # If found, save the table index
            indices.append(idx)
    # extract summary table related to CELLS walls 
    
    # !!! Hardcoded        
    df_ufactor = pd.DataFrame(df[32])
    
    # recompute averaged U-factor    
    numerator = numerator + ((df_ufactor['U-Factor with Film [W/m2-K]']*df_ufactor['Gross Area [m2]']).sum())
    denominator = denominator + df_ufactor['Gross Area [m2]'].sum()
    U_factor = numerator/denominator
    # convert to W -> kW
    return U_factor/1000, indices

In [56]:
def modify_setpoint(idf, new_setpoint, name_SIA, modified_hours=list(range(9, 18)), verbose = 'q'):
    for entity  in idf.idfobjects:
        for field in idf.idfobjects[entity]:
            if 'Name' in field.objls: # check field with names only
                if name_SIA in field.Name: #modify fields where modified SIA file appears
                    for hours in modified_hours:
                        #if hours in modified_hours:
                        # Replace the value with new setpoint
                            setattr(field, f"Value_{hours}", new_setpoint)
                    if verbose != 'q':
                        print(field)

In [88]:
def modify_capacity(new_capacity):
    print(new_capacity)

In [57]:
U_Factor, _ = extract_U_factor(r'output_data\idf1_tbl.htm')

In [58]:
SIA_heating_schedule = '10003_-50_12_21'
SIA_cooling_schedule = '10003_100_28_26'

In [59]:
buildings['thermal_transmittance_signature_kW_m2_K'] = U_Factor

In [92]:
# Set building parameters
# you can as well define your district from a csv file instead of reading the database
reader = QBuildingsReader()

qbuildings_data = reader.read_csv(buildings_filename=r'.\data\buildings.csv', nb_buildings=1)

# Select weather data
cluster = {'Location': 'Geneva', 'Attributes': ['I', 'T', 'W'], 'Periods': 10, 'PeriodDuration': 24}

# Set scenario
scenario = dict()
scenario['Objective'] = 'TOTEX'
scenario['name'] = 'totex'
scenario['exclude_units'] = ['Battery', 'NG_Cogeneration', 'PV']
scenario['enforce_units'] = []

# Initialize available units and grids
grids = infrastructure.initialize_grids()
units = infrastructure.initialize_units(scenario, grids)

# Set method options
method = {}

# Run optimization
reho = reho(qbuildings_data=qbuildings_data, units=units, grids=grids, cluster=cluster, scenario=scenario, method=method, solver='gurobi')

reho.single_optimization()
reho.save_results(format=['xlsx', 'pickle'], filename='reho_1')


Key egid not in the dictionary
Key class not in the dictionary
Key period not in the dictionary
Key capita_cap not in the dictionary
Key height_m not in the dictionary
Key count_floor not in the dictionary
Key source_heating not in the dictionary
Key source_hotwater not in the dictionary
Key x not in the dictionary
Key y not in the dictionary
Key z not in the dictionary
Key geometry not in the dictionary
Key transformer not in the dictionary
Key energy_heating_signature_kWh_y not in the dictionary
Key energy_cooling_signature_kWh_y not in the dictionary
Key energy_hotwater_signature_kWh_y not in the dictionary
Key energy_el_kWh_y not in the dictionary
Key roof_annual_irr_kWh_y not in the dictionary
Key facade_annual_irr_kWh_y not in the dictionary
No geometry in the dataframe
Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  AN

In [70]:
display(reho.results.items())

dict_items([('totex', {0: {'df_Performance':              Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        0.0  1013.06396  , 'df_Annuals':                                             Demand_MWh  Supply_MWh
Layer       Hub                

In [86]:
new_capacity = reho.results['totex'][0]['df_Unit']['Units_Mult'].sum()

0.20211404213

In [ ]:
# generate updated idf object 
modify_capacity(new_capacity)

In [120]:
# main loop
new_capacity = 'autosize'
for i in range(0,10):
    ### EnergyPlus Simulation
    #update idf object with new idf_file_path
    idf.idfobjects['Sizing:System'][0]['Cooling_Design_Capacity'] = new_capacity
    idf = IDF(idf_file_path, weather_file_path)
    
    # generate new EPlus results
    idf.run(readvars=True,output_directory=idf_results_path, annual=True, verbose='q', output_prefix='idf1_')
    
    ### EnergyPlus - REHO interface
    # extract U-Factor from EPlus results
    U_Factor, _ = extract_U_factor(r'output_data\idf1_tbl.htm')
    display(U_Factor)
    # new_thermal_capacity = extract_thermal_capacity()
    new_heat_capacity = 120
    
    # Update REHO object   
    reho.buildings_data['Building1']['U_h'] = U_Factor
    reho.buildings_data['Building1']['Heat_capacity'] = new_heat_capacity*1000 # from kW to W 

    ### REHO optimization
    # Run REHO optimization
    reho.single_optimization()
    
    # Extract the installed capacity
    new_capacity = reho.results['totex'][0]['df_Unit']['Units_Mult'].sum()

0.00014987943262411351

Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        

0.00014987943262411351

Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        

0.00014987943262411351

...

Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        

0.00014987943262411351

Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        

0.00014987943262411351

Gurobi 11.0.0:   tech:nodefilestart = 0.5
Gurobi 11.0.0: optimal solution; objective 1013.06396
464 simplex iteration(s)
1 branching node(s)
absmipgap=4.54747e-13, relmipgap=0
             Costs_op   Costs_inv  ANN_factor  Costs_grid_connection  \
Hub                                                                    
Building1  824.432859  188.629623    0.061157                    0.0   
Network    824.432859  188.629623    0.061157                    0.0   

           Costs_rep  Costs_ft      GWP_op  GWP_constr  EMOO_CAPEX  EMOO_OPEX  \
Hub                                                                             
Building1        0.0         0  320.311631     0.33537         NaN        NaN   
Network          0.0         0  320.311631     0.33537         0.0        0.0   

           EMOO_TOTEX  EMOO_GWP  EMOO_grid   Objective  
Hub                                                     
Building1         NaN       NaN        NaN         NaN  
Network           0.0       0.0        

In [114]:
idf.idfobjects['Sizing:System'][0].objls

['key',
 'AirLoop_Name',
 'Type_of_Load_to_Size_On',
 'Design_Outdoor_Air_Flow_Rate',
 'Central_Heating_Maximum_System_Air_Flow_Ratio',
 'Preheat_Design_Temperature',
 'Preheat_Design_Humidity_Ratio',
 'Precool_Design_Temperature',
 'Precool_Design_Humidity_Ratio',
 'Central_Cooling_Design_Supply_Air_Temperature',
 'Central_Heating_Design_Supply_Air_Temperature',
 'Type_of_Zone_Sum_to_Use',
 '100_Outdoor_Air_in_Cooling',
 '100_Outdoor_Air_in_Heating',
 'Central_Cooling_Design_Supply_Air_Humidity_Ratio',
 'Central_Heating_Design_Supply_Air_Humidity_Ratio',
 'Cooling_Supply_Air_Flow_Rate_Method',
 'Cooling_Supply_Air_Flow_Rate',
 'Cooling_Supply_Air_Flow_Rate_Per_Floor_Area',
 'Cooling_Fraction_of_Autosized_Cooling_Supply_Air_Flow_Rate',
 'Cooling_Supply_Air_Flow_Rate_Per_Unit_Cooling_Capacity',
 'Heating_Supply_Air_Flow_Rate_Method',
 'Heating_Supply_Air_Flow_Rate',
 'Heating_Supply_Air_Flow_Rate_Per_Floor_Area',
 'Heating_Fraction_of_Autosized_Heating_Supply_Air_Flow_Rate',
 'Heating_F